In [622]:
import torch
from sklearn.preprocessing import StandardScaler
import torchvision.transforms as T
from torch import nn
from umap.umap_ import UMAP
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
import numpy as np
from sklearn.cluster import KMeans

Redefine the model architecture but instead of returning the last layer, we return the penultimate layer

In [623]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = torch.nn.Sequential(
            nn.Conv2d(3,9,(5,5)),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Conv2d(9,18,(5,5)),
            nn.ReLU(),
            nn.MaxPool2d((2,2)),
            nn.Flatten(1,-1),
            nn.Linear(288,64)
            )
        self.relu = nn.ReLU()
        self.l = nn.Linear(64,5)

    def forward(self, x):
        x = self.net(x)
        return x


Load the model and the training data

In [639]:
model = LeNet()
model.load_state_dict(torch.load('/Users/sanjay/Desktop/Python Projects/AI/George/model.pt', weights_only=True))
model.eval()
train_images = torch.load('/Users/sanjay/Desktop/Python Projects/AI/George/train_images.pt')
train_labels = torch.load('/Users/sanjay/Desktop/Python Projects/AI/George/train_labels.pt')

/var/folders/bn/hglmht2s7y19n14x7s3vfm2m0000gn/T/ipykernel_99750/3168465676.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_images = torch.load('/Users/sanjay/Desk

Make sure the training data was not corrupted

In [625]:
train_images.shape

torch.Size([48004, 3, 28, 28])

Sort all the images by labels, and keep track of the index for later use

In [641]:

transform = T.Compose([
    T.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5))
])
output = [[],[],[],[],[]]
indices = [[],[],[],[],[]]
with torch.no_grad():
    for index,(x,label) in enumerate(zip(train_images, train_labels)):
        output[label].append(model(transform(torch.unsqueeze(x,0)))[0])
        indices[label].append(index)

In [642]:
print(output[0][0])

tensor([ -4.4966,   9.8564,   6.3036,  -3.9335,  -4.5853,   3.6010,  -1.1261,
         -1.3643,  -0.1630,   4.0448,   3.3420,  -2.6474,  -5.0475,  -7.2159,
         -0.3373,   7.3108,  -1.0397,  11.7725,   8.1777,   3.1699,   7.2453,
          3.9548,  -0.4841,  -2.1406,  -3.0027,   3.2276,   2.8054,   8.0212,
          4.6876,   9.9327,  -1.5293,  -5.5020,  -2.5677, -12.5115,   5.3602,
          0.6122,   9.4569,  -0.8947,   4.2664,   9.3138,   8.5346,  -1.5404,
         -3.2625,  -2.3400,  -3.3974,   5.2827, -12.6329,  -1.6231,   0.5767,
          6.5745,  -3.2901,  -1.0705,  -0.0554,   5.1487,  -4.8576,  -3.0579,
          2.5994,  -1.3498,  -5.7117,  -0.9312,  -0.3212,   2.3399,  -0.8020,
          8.4741])


Create a reducer to transform n-dimensional tensors in 2-dimensional tensors

In [629]:
reducer1 = UMAP()

Normalize the data to aid with dimensionality reducing speeds

In [630]:
s1 = StandardScaler().fit_transform(output[0])
s2 = StandardScaler().fit_transform(output[1])
s3 = StandardScaler().fit_transform(output[2])
s4 = StandardScaler().fit_transform(output[3])
s5 = StandardScaler().fit_transform(output[4])

Reduce the dimensionality of the data

In [ ]:
e1 = reducer1.fit_transform(s1)
e2 = reducer1.fit_transform(s2)
e3 = reducer1.fit_transform(s3)
e4 = reducer1.fit_transform(s4)
e5 = reducer1.fit_transform(s5)
e1.shape

(10133, 2)

For every single group of labels, figure out the number of clusters and their centers by trying out 2-10 clusters and seeing which one results in the highest silhouette score. Then, for each cluster that we found, break it into F subcluster, F in this case being 10. Record all the centers of each cluster

In [633]:
e = [e1,e2,e3,e4,e5]
bk = [0,0,0,0,0]
f = 10
centers = []
for i in range(5):
    bs = 0
    for k in range(2,10):
        gm = GaussianMixture(k).fit_predict(e[i])
        score = silhouette_score(e[i], gm)
        if score > bs:
            bs = score
            bk[i] = k
    predictor = GaussianMixture(bk[i]).fit(e[i])
    temp = []
    labels = predictor.predict(e[i])
    for cluster_value in np.unique(labels):
        cluster = e[i][labels == cluster_value]
        subclusters = GaussianMixture(f).fit(cluster).means_
        for subsubclusters in subclusters:
            temp.append(subsubclusters)
    centers.append(temp)

Label each image according to which subcluster it belongs to. We add num to make sure that images that belong to different labels dont have the same subcluster number

In [635]:
labels = np.array([])
num = 0
for i in range(5):
    labels = np.concatenate((labels, KMeans(len(centers[i]), init = centers[i]).fit_predict(e[i]) + num))
    num += len(centers[i])

Sort the array of subcluster labels in order using the indicies we recorded earlier

In [636]:
index = np.array([])
for i in range(5):
    index = np.concatenate((index, indices[i]))

idx = np.argsort(index)
labels = np.array(labels)[idx]
index = np.array(index)[idx]

Save the data

In [637]:
df = pd.DataFrame({'Labels': labels})
df.to_csv('groups.csv', index= False)